### Creating Database Files

This code creates a text file containing a json of all the institutions in the assist.org database.

In [ ]:
import pandas as pd
import requests
import json

api_url = "https://assist.org/api/institutions"
response = requests.get(api_url).json()

dfAll = pd.DataFrame(columns=['id', 'name', 'code', 'isCCC'])

for i in response:
    entry = {'id': i['id'], 'name': i['names'][0]['name'], 'code': i['code'], 'isCCC': i['isCommunityCollege']}
    dfAll = pd.concat([dfAll, pd.DataFrame([entry])], ignore_index = True)

dfAll_json = dfAll.to_json(orient='records')

with open('institutions.txt', 'w') as f:
    f.write(dfAll_json)
    f.close()

This code creates a text file containing a json of all the institutions with agreements to De Anza College, as well as one only containing the UCs.

In [17]:
api_url = "https://assist.org/api/institutions/113/agreements"
response = requests.get(api_url).json()

dfAgree = pd.DataFrame(columns=['id', 'name', 'code', 'isCCC'])

for i in response:
    entry = {'id': i['institutionParentId'], 'name': i['institutionName'], 'code': i['code'], 'isCCC': i['isCommunityCollege'], 'years': i['receivingYearIds']}
    dfAgree = pd.concat([dfAgree, pd.DataFrame([entry])], ignore_index = True)

dfAgree_json = dfAgree.to_json(orient='records')

with open('all_agreements.txt', 'w') as f:
    f.write(dfAgree_json)
    f.close()

uc_json = dfAgree[dfAgree.name.str.contains("University of California")].to_json(orient='records')

with open('uc_agreements.txt', 'w') as f:
    f.write(uc_json)
    f.close()


This code creates a text file containing all keys for the agreements between the UC's and De Anza College.

In [24]:
dfUC = dfAgree[dfAgree.name.str.contains("University of California")]
dfKeys = pd.DataFrame(columns=['school', 'label', 'key'])

for i, d in dfUC.iterrows():
    home = "113"
    destination_id = d['id']
    destination_name = d['name']
    years = d['years']
    year = max(years)
    
    api_url = f"https://assist.org/api/agreements?receivingInstitutionId={destination_id}&sendingInstitutionId={home}&academicYearId={year}&categoryCode=major"
    response = requests.get(api_url).json()
    
    for i in response['reports']:
        entry = {'school': destination_name, 'label': i['label'], 'key': i['key']}
        dfKeys = pd.concat([dfKeys, pd.DataFrame([entry])], ignore_index = True)

keys_json = dfKeys.to_json(orient='records')

with open('uc_agreement_keys.txt', 'w') as f:
    f.write(keys_json)
    f.close()

This code downloads each pdf file for each agreement key provided.

In [28]:
for i, d in dfKeys.iterrows():
    key = d['key']
    api_url = f"https://assist.org/api/artifacts/{key}"
    response = requests.get(api_url)
    with open(f'./pdfs/{key}.pdf', 'wb') as f:
        f.write(response.content)
        f.close()